In [2]:
# Read dataset - tiny shakespeare
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# unique characters in training data
chars = sorted(list(set(text)))
vocab_size = len(chars) # possible elemnts/characters of out sequences (which model can see and imits)
print("used chars...")
print(''.join(chars))
print(f"number of unique chars in data: {vocab_size}")

used chars...

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
number of unique chars in data: 65


### Basic Way How to Proceed Character-Level Tokenization and Prepare Training Data
#### Production-grade solutions: 
sentencepiece: https://github.com/google/sentencepiece <br>
tiktoken: https://github.com/openai/tiktoken <br>

In [9]:
# let's create mapping from strings to integers and vice versa
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers and retunrs a string 

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
# now let's encode the entire text dataset and store it into torch.Tensor structure --> translation characters into integers
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
# as usual we have to split up the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
# create size of one training chunk
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [24]:
x = train_data[:block_size]
y = train_data[1:block_size+1] # shift by 1 to the right to alling context integers with targets
for t in range(block_size):
    context = x[:t+1] # it is given as a context for next char generation
    target = y[t] # this is a target for prediction (based on context sequence)
    print(f"when inputting is {context} the target is {target}")

when inputting is tensor([18]) the target is 47
when inputting is tensor([18, 47]) the target is 56
when inputting is tensor([18, 47, 56]) the target is 57
when inputting is tensor([18, 47, 56, 57]) the target is 58
when inputting is tensor([18, 47, 56, 57, 58]) the target is 1
when inputting is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when inputting is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when inputting is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [28]:
# Batch: during training there is many batches of multiple chinks of the text
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('-------------')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the t

### Implement First Neural Net

In [60]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # B, T, C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # we have to reshape dimensions because torch.functional.cross_entropy expects order B, C, T instead of B, T, C
            logits = logits.view(B*T, C) # flatten first and second dimension
            targets = targets.view(B*T) # or -1 
            loss = F.cross_entropy(logits, targets) # loss fuction for classification tasks

        return logits, loss # logits are scores for next characters sequence
     
    def generate(self, idx, max_new_tokens): # expanding generation
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1,:] # becomes (B, C) because T is obviously "fixed"
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb) # Why it is returning output withnout training? Initial weights doing their job?
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([256, 65])
tensor(4.7203, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [67]:
# now let's train the model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-03) # take gradients and update them

In [79]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data 
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4670538902282715


In [82]:
print(decode(m.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


MARG wnd.
Ers abed the; afis ffit!
Gersiunthesthy brs mune trdined: wesal, wingog itr w mndres thambathaiche Yedirur tequ ps t ld.
Hathelyon wannom speasatiath,
O, ge itheng KIONCY sun b' had the; ly ybburere be;
F halupure 's y'lontthailllat aucesintheryse ucothe blly osy o he.
Beeseallthicheay ath tthelll suond; y RIZWir lyomexcl
The ou irthtur,
Whour
Sonshive h aus LLe me y.
Matrne sh'd t;
Ant 


### Self-Attention -- The Mathematical trick

In [90]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

torch.Size([4, 8, 2])

In [93]:
xbow = torch.zeros((B, T, C)) # back of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0) # averaging ov the 0-th dimension, which is time here because B is "fixed"

In [91]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [94]:
xbow[0] # like an expanding mean, so that first is as same as x[0] and the last item is mean over the whole batch

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [98]:
# Since 2x for loop is not efficient at all, let's procced a matrix multiplication (our mathematical trick) 
torch.manual_seed(1337)
a = torch.tril( torch.ones(3, 3) )
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b # this is matrix multiplication, be aware of difference in comparions to the dot product and Hadamard product


In [99]:
c

tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])

In [101]:
# Let's apply it...
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C)  ----> (B, T, C)
torch.allclose(xbow, xbow2) # test if the two are close together

True

In [102]:
# Rewrite it again incorporating Softmax normalization layer
tril = torch.tril(torch.ones(T,T)) 
wei = torch.zeros((T,T)) # interaction strength, by default 0 but actually there are data-dependent
wei = wei.masked_fill(tril==0, float('-inf')) # tokens from the past are not aggregatted (for -inf value), it is implementation that future (tokens) can't communicate with the past
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

### Understanding of Self-Attention
Attention is a communication mechanism which can be seen as nodes in a directed graphs looking at each other and aggregating with a weigted sum from all nodes that point to them, with data-dependent weights.

In [116]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
# every single node/token emits 2 vectors: query (what am I looking for) and key (what do I contain), affinities are given byt product of keys and queries
# let's have a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
# so far there is no communication happens yet
# communication starts now
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf')) # this keeps time causality where the future is unknown or simply said, time (the vector) runs just in one direction
wei = F.softmax(wei, dim=-1)

v = value(x) # the communication to other nodes/tokens
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [117]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

Self-Attention means that keys, queries and values are coming from the same source x, so that the nodes are self-attending.
Cross-Attention is used when tere are separate sources of nodes we would like to pull the infromation.
"Scaling" the attention with the square root of number of heads: Q, K are controlled by theif vaiance.